In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Combined_Prior_to_Full_Script_Run_01-09-2022.csv")
print(df.shape)
df.head()

(299364, 6)


,Population,Scrape_Date,STATE-COUNTY,Month,Year,DayOfWeek
0,172.0,2020-03-10,AL-Autauga,3,2020,Tuesday
1,173.0,2020-03-11,AL-Autauga,3,2020,Wednesday
2,174.0,2020-03-12,AL-Autauga,3,2020,Thursday
3,180.0,2020-03-13,AL-Autauga,3,2020,Friday
4,180.0,2020-03-14,AL-Autauga,3,2020,Saturday


In [3]:
df['STATE-COUNTY'].nunique()

1015

In [4]:
original = pd.read_csv("RawData_03-10-21_to_10-07-21_Generated_01-09-2022.csv")
original.head()

,Unnamed: 0,Population,Scrape_Date,STATE-COUNTY,Month,Year,DayOfWeek
0,31,172.0,2020-03-10,AL-Autauga,3,2020,Tuesday
1,32,173.0,2020-03-11,AL-Autauga,3,2020,Wednesday
2,33,174.0,2020-03-12,AL-Autauga,3,2020,Thursday
3,34,180.0,2020-03-13,AL-Autauga,3,2020,Friday
4,35,180.0,2020-03-14,AL-Autauga,3,2020,Saturday


In [5]:
original['STATE-COUNTY'].nunique()

415

In [7]:
df = df[df['STATE-COUNTY'].isin(original['STATE-COUNTY'].unique())]
df['STATE-COUNTY'].nunique()

415

In [8]:
df['Scrape_Date'] = pd.to_datetime(df['Scrape_Date'])

/home/matt/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263550 entries, 0 to 299362
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Population    249201 non-null  float64       
 1   Scrape_Date   263550 non-null  datetime64[ns]
 2   STATE-COUNTY  263550 non-null  object        
 3   Month         263550 non-null  int64         
 4   Year          263550 non-null  int64         
 5   DayOfWeek     263550 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 14.1+ MB


## Script Imports

In [10]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import glob
import numpy as np
import re


## Rest of Script

In [11]:

## Summarizing Daily Jail Populations
## This is the second tab of the saved worksheet.
jail_totals = df.groupby('Scrape_Date')['Population'].agg(['count', 'sum'])
jail_totals.columns = ['Count_Of_Jails','Total_Jail_Population']
jail_totals['Seven_Day_Rolling_Average'] = jail_totals['Total_Jail_Population'].rolling(7).mean()


## First Day Available of Each Month
## Ensure Sorted from earliest to most recent
df = df.sort_values(by=['STATE-COUNTY', 'Scrape_Date'])

first_days = df.groupby(['STATE-COUNTY', 'Month', 'Year']).head(1)
first_days = first_days.copy() ##Create copy to avoid warning since first_days is a slice of df
first_days['Scrape_Date'] = "FirstDay" + first_days['Month'].astype(str) + "-" + first_days['Year'].astype(str)

In [12]:
## First Monday of Each Month

mondays = df[df.DayOfWeek=='Monday']
first_mondays = mondays.groupby(['STATE-COUNTY', 'Month', 'Year']).head(1)

In [13]:
most_recent = df.groupby(['STATE-COUNTY']).tail(1)

In [14]:
## Verify DataFrame Correctly Sorted by Facility/Date

assert(all(df == df.sort_values(by=['STATE-COUNTY', 'Scrape_Date'])))
## Ensure there is one per Facility
# assert(len(most_recent) == df.STATE-COUNTY.nunique()) #Deprecated; would need to populate state/county by splitting new state-county field which is true by definition.


## March 10th

march_10 = df[df.Scrape_Date == pd.to_datetime("03-10-2020")]

## Non Script: Preview Outputs

In [15]:
first_mondays.head()

,Population,Scrape_Date,STATE-COUNTY,Month,Year,DayOfWeek
6,178.0,2020-03-16,AL-Autauga,3,2020,Monday
27,139.0,2020-04-06,AL-Autauga,4,2020,Monday
55,156.0,2020-05-04,AL-Autauga,5,2020,Monday
83,174.0,2020-06-01,AL-Autauga,6,2020,Monday
118,170.0,2020-07-06,AL-Autauga,7,2020,Monday


In [16]:
most_recent.head()

,Population,Scrape_Date,STATE-COUNTY,Month,Year,DayOfWeek
267774,165.000000,2021-11-30,AL-Autauga,11,2021,Tuesday
268078,215.000000,2021-11-30,AL-Chilton,11,2021,Tuesday
267945,47.769231,2021-11-30,AL-Clay,11,2021,Tuesday
268276,52.000000,2021-11-30,AL-Cleburne,11,2021,Tuesday
268079,125.000000,2021-11-30,AL-Coffee,11,2021,Tuesday


In [17]:
march_10.head()

,Population,Scrape_Date,STATE-COUNTY,Month,Year,DayOfWeek
0,172.0,2020-03-10,AL-Autauga,3,2020,Tuesday
577,212.0,2020-03-10,AL-Chilton,3,2020,Tuesday
1154,39.0,2020-03-10,AL-Clay,3,2020,Tuesday
1731,84.0,2020-03-10,AL-Cleburne,3,2020,Tuesday
2308,127.0,2020-03-10,AL-Coffee,3,2020,Tuesday


## Continue Script

In [18]:
## First Mondays Summary
first_monday_summary = first_mondays.groupby(['STATE-COUNTY', 'Month', 'Year']).head(1)
first_monday_summary['Scrape_Date'] = "FirstMondayAvailable" + first_monday_summary['Month'].astype(str) + "-" + first_monday_summary['Year'].astype(str)

# Stitching Together the Final Summary
pre_pivot = pd.concat([march_10, first_mondays, most_recent, first_monday_summary, first_days], ignore_index=True)
# Remove Potential Duplicates (First Mondays and Most Recent could include duplicates)
pre_pivot = pre_pivot.drop_duplicates(subset=['STATE-COUNTY', 'Scrape_Date'])
final = pre_pivot.pivot(index="STATE-COUNTY", columns="Scrape_Date", values="Population")

In [19]:
## Save Transformed Output to Excel

today = datetime.datetime.now().strftime('%m-%d-%Y') #Create string of today's date
max_scrape_date = max([col for col in final.columns if re.match("\d{4}-\d{2}-\d{2}", str(col))])
min_scrape_date = min([col for col in final.columns if re.match("\d{4}-\d{2}-\d{2}", str(col))])

filename = "Jail_Summaries_{}_to_{}_Generated_{}.xlsx".format(min_scrape_date, max_scrape_date, today)
print('Saving to: {}'.format(filename))

with pd.ExcelWriter(filename) as writer1:
    to_save = [(final, 'FirstMondays'),(jail_totals, 'DailyJailTotals')]
    #final.to_excel(writer1, sheet_name = 'FirstMondays')
    #jail_totals.to_excel(writer1, sheet_name = 'DailyJailTotals')
    #Fix Column Widths
    for df, sheetname in to_save:
        df.to_excel(writer1, sheet_name = sheetname)
        worksheet = writer1.sheets[sheetname]  # pull worksheet object

        #Update Sheet Formatting - ensures columns are expanded to width of longest element rathen than collapsed.
        for idx, col in enumerate(df):  # loop through all columns
            series = df[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 12  # adding a little extra space
            worksheet.set_column(idx, idx, max_len)  # set column width



Saving to: Jail_Summaries_2020-03-10 00:00:00_to_2021-11-30 00:00:00_Generated_01-11-2022.xlsx


In [20]:
print(final.shape)
final.head()

(415, 72)


Scrape_Date,2020-03-10 00:00:00,2020-03-16 00:00:00,2020-04-06 00:00:00,2020-05-04 00:00:00,2020-06-01 00:00:00,2020-07-06 00:00:00,2020-08-03 00:00:00,2020-09-07 00:00:00,2020-10-05 00:00:00,2020-11-02 00:00:00,...,FirstMondayAvailable5-2020,FirstMondayAvailable5-2021,FirstMondayAvailable6-2020,FirstMondayAvailable6-2021,FirstMondayAvailable7-2020,FirstMondayAvailable7-2021,FirstMondayAvailable8-2020,FirstMondayAvailable8-2021,FirstMondayAvailable9-2020,FirstMondayAvailable9-2021
STATE-COUNTY,,,,,,,,,,,,,,,,,,,,,
AL-Autauga,172.0,178.0,139.0,156.0,174.0,170.0,147.0,160.0,160.0,185.0,...,156.0,163.0,174.0,166.0,170.0,158.0,147.0,159.0,160.0,150.0
AL-Chilton,212.0,207.0,152.0,172.0,175.0,184.0,158.0,153.0,188.0,184.0,...,172.0,177.0,175.0,211.0,184.0,198.0,158.0,216.0,153.0,194.0
AL-Clay,39.0,42.0,31.0,28.0,31.0,34.0,30.0,32.0,36.0,33.0,...,28.0,48.0,31.0,54.0,34.0,60.0,30.0,44.0,32.0,54.0
AL-Cleburne,84.0,85.0,75.0,62.0,73.0,61.0,66.0,69.0,70.0,69.0,...,62.0,53.0,73.0,63.0,61.0,70.0,66.0,71.0,69.0,72.0
AL-Coffee,127.0,127.0,66.0,61.0,58.0,76.0,75.0,91.0,79.0,83.0,...,61.0,135.0,58.0,143.0,76.0,146.0,75.0,145.0,91.0,158.0


## Compare to Previous Output

In [29]:
# prior = pd.read_excel("Jail_Summaries_2020-03-10_to_2021-10-07_Generated_01-09-2022.xlsx")
prior = pd.read_excel("Jail_Summaries_as_of_10-08-2021.xlsx")
prior = prior.set_index("STATE-COUNTY")
prior.head(2)

,2020-03-10 00:00:00,2020-03-16 00:00:00,2020-04-06 00:00:00,2020-05-04 00:00:00,2020-06-01 00:00:00,2020-07-06 00:00:00,2020-08-03 00:00:00,2020-09-07 00:00:00,2020-10-05 00:00:00,2020-11-02 00:00:00,...,FirstMondayAvailable07-2020,FirstMondayAvailable07-2021,FirstMondayAvailable08-2020,FirstMondayAvailable08-2021,FirstMondayAvailable09-2020,FirstMondayAvailable09-2021,FirstMondayAvailable10-2020,FirstMondayAvailable10-2021,FirstMondayAvailable11-2020,FirstMondayAvailable12-2020
STATE-COUNTY,,,,,,,,,,,,,,,,,,,,,
AL-Autauga,172,178.0,139.0,156.0,174.0,170.0,147.0,160.0,160.0,185.0,...,170.0,158.0,147.0,159.0,160.0,150.0,160.0,158.0,185.0,184.0
AL-Chilton,212,207.0,152.0,172.0,175.0,184.0,158.0,153.0,188.0,184.0,...,184.0,198.0,158.0,216.0,153.0,194.0,188.0,218.0,184.0,200.0


In [30]:
diff = prior - final
diff.head()

,2020-03-10 00:00:00,2020-03-16 00:00:00,2020-04-06 00:00:00,2020-05-04 00:00:00,2020-06-01 00:00:00,2020-07-06 00:00:00,2020-08-03 00:00:00,2020-09-07 00:00:00,2020-10-05 00:00:00,2020-11-02 00:00:00,...,FirstMondayAvailable5-2020,FirstMondayAvailable5-2021,FirstMondayAvailable6-2020,FirstMondayAvailable6-2021,FirstMondayAvailable7-2020,FirstMondayAvailable7-2021,FirstMondayAvailable8-2020,FirstMondayAvailable8-2021,FirstMondayAvailable9-2020,FirstMondayAvailable9-2021
STATE-COUNTY,,,,,,,,,,,,,,,,,,,,,
AL-Autauga,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL-Chilton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL-Clay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL-Cleburne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL-Coffee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
pd.options.display.max_rows = None

In [32]:
diff.sum().sum()

0.0

In [33]:
diff.sum()

2020-03-10 00:00:00            0.0
2020-03-16 00:00:00            0.0
2020-04-06 00:00:00            0.0
2020-05-04 00:00:00            0.0
2020-06-01 00:00:00            0.0
2020-07-06 00:00:00            0.0
2020-08-03 00:00:00            0.0
2020-09-07 00:00:00            0.0
2020-10-05 00:00:00            0.0
2020-11-02 00:00:00            0.0
2020-12-07 00:00:00            0.0
2021-01-04 00:00:00            0.0
2021-02-01 00:00:00            0.0
2021-03-01 00:00:00            0.0
2021-04-05 00:00:00            0.0
2021-05-03 00:00:00            0.0
2021-06-07 00:00:00            0.0
2021-07-05 00:00:00            0.0
2021-07-16 00:00:00            0.0
2021-08-01 00:00:00            0.0
2021-08-02 00:00:00            0.0
2021-08-16 00:00:00            0.0
2021-08-19 00:00:00            0.0
2021-08-23 00:00:00            0.0
2021-08-26 00:00:00            0.0
2021-08-27 00:00:00            0.0
2021-08-28 00:00:00            0.0
2021-08-29 00:00:00            0.0
2021-09-02 00:00:00 